In [1]:
import requests 
from bs4 import BeautifulSoup
import html
import pandas as pd, numpy as np 
from io import StringIO
import math  
import sqlite3, sqlalchemy 
import re 

pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 15)  

In [2]:
def clean_player(name): 
    i = name.find('*')   
    name = name[:i] if i > -1 else name 
    return name 

def load_soup(URL): 
    r = requests.get(URL) 
    soup = BeautifulSoup(r.content, 'html') 

    return soup 

def select_team(group):
    value = group.head(1) 
    if 'Team' in value.columns:
        if len(group) > 1: 
            value['Team'] = "" 
            for i in range(1, len(group)):
                value['Team'] += group.iloc[i]['Team'] 
                value['Team'] += '-' if i < len(group)-1 else ''  
    else: 
        if len(group) > 1: 
            value['Tm'] = "" 
            for i in range(1, len(group)):
                value['Tm'] += group.iloc[i]['Tm'] 
                value['Tm'] += '-' if i < len(group)-1 else ''  
    return value 

def html_df(soup, key, value, remove_cols=['Rk'], drop_cols=True, drop_lvl=False): 
    stat_df = soup.find('table', attrs={key:value}) 
    stat_df = pd.read_html(StringIO(str(stat_df))) 
    stat_df = pd.DataFrame(stat_df[0]) 
    if drop_lvl: 
        stat_df.columns = stat_df.columns.droplevel() 
    stat_df = stat_df.groupby('Player').apply(lambda x: select_team(x)).reset_index(drop=True) 
    stat_df['Player'] = stat_df['Player'].apply(lambda x: clean_player(x)) 
    if drop_cols==True:  
        stat_df = stat_df.drop(remove_cols, axis=1) 

    return stat_df 

def combine_dfs(dfs):
    curr = None
    for name in dfs.keys():
        if curr is None: 
            curr = dfs[name]
        else:
            diff = dfs[name].columns.difference(curr.columns).tolist()
            diff.append('Player') 
            curr = pd.merge(curr, dfs[name].loc[:,diff], on = 'Player') 
    return curr  

def clean_if_dirty(master):
    filtered = master[master['Player']=='Player']
    pos = filtered.index
    master = master.drop(pos, axis=0) 
    master['Player'] = master['Player'].apply(lambda x: clean_player(x)) 
    master = master.dropna(axis=1, how='all')   
    if ('Tm' in master.columns) & ('Team' in master.columns): 
        master = master.drop('Tm', axis=1) 
         
    return master 

def mod_types_pct(master): 
    for col_name in master.columns: 
        if col_name not in ['Player', 'Team', 'Tm', 'Pos', 'Awards']:  
            master[col_name] = master[col_name].astype("float64")
        if col_name in ['FG%', '3P%', 'FT%', 'eFG%', 'TS%', '2P%']:
            master[col_name] *= 100 
    return master

def process_totals(totals, names): 
    if '3P' in totals.columns:  
        totals = totals.loc[:,['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB']] 
    elif 'BLK' in totals.columns: 
        totals = totals.loc[:,['PTS', 'TRB', 'AST', 'STL', 'BLK', 'PF', 'MP', 'FG', 'FGA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB']] 
    elif 'TRB':
        totals = totals.loc[:,['PTS', 'TRB', 'AST', 'PF', 'FG', 'FGA', '2P', '2PA', 'FT', 'FTA']] 
    else: 
        totals = totals.loc[:,['PTS', 'AST', 'PF', 'FG', 'FGA', '2P', '2PA', 'FT', 'FTA']] 
    totals.columns = 'T' + totals.columns 
    totals['Player'] =  names 

    return totals 

def process_averages(master):
    if 'TOV' in master.columns: 
        new = master.rename(columns={'G':'GP', 'MP':'MPG', 'ORB':'ORPG', 'DRB':'DRPG', 'TRB':'RPG', 'AST':'APG', 'STL':'SPG', 'BLK':'BPG', 'TOV':'ToPG', 'PTS':'PPG'}, errors="raise")
    elif 'BLK' in master.columns: 
        new = master.rename(columns={'G':'GP', 'MP':'MPG', 'ORB':'ORPG', 'DRB':'DRPG', 'TRB':'RPG', 'AST':'APG', 'STL':'SPG', 'BLK':'BPG', 'PTS':'PPG'}, errors="raise") 
    elif 'TRB' in master.columns: 
        new = master.rename(columns={'G':'GP', 'TRB':'RPG', 'AST':'APG', 'PTS':'PPG'}, errors="raise") 
    else: 
        new = master.rename(columns={'G':'GP', 'AST':'APG','PTS':'PPG'}, errors="raise") 

    return new  

In [3]:
engine = sqlalchemy.create_engine('sqlite:///../../DB/ballbase.db') 

start_ssn = 1965     
stop_ssn = 1973                     

for season in range(start_ssn, stop_ssn): 
    
    URL_averages = 'https://www.basketball-reference.com/leagues/NBA_' + str(season+1) + '_per_game.html' 
    soup_averages = load_soup(URL_averages) 
    averages = html_df(soup_averages, 'id', 'per_game_stats', ['Rk'])  

    URL_totals = 'https://www.basketball-reference.com/leagues/NBA_' + str(season+1) + '_totals.html' 
    soup_totals = load_soup(URL_totals) 
    totals = html_df(soup_totals, 'id', 'totals_stats', ['Rk']) 
    totals = process_totals(totals, averages['Player'])   
    
    if season >= 1973: 
        URL_per100 = 'https://www.basketball-reference.com/leagues/NBA_' + str(season+1) + '_per_poss.html' 
        soup_per100 = load_soup(URL_per100) 
        per100 = html_df(soup_per100, 'id', 'per_poss_stats', ['Rk', 'Unnamed: 29']) 

    URL_advanced = 'https://www.basketball-reference.com/leagues/NBA_' + str(season+1) + '_advanced.html'  
    soup_advanced = load_soup(URL_advanced) 
    advanced = html_df(soup_advanced, 'id', 'advanced_stats', ['Rk', 'Unnamed: 19', 'Unnamed: 24'])   

    if season >= 1973: 
        dfs = {"averages": averages, "totals": totals, "per100": per100, "advanced": advanced} 
    else: 
        dfs = {"averages": averages, "totals": totals, "advanced": advanced} 

    season_str = str(season) + '_' + str(season+1)[2:]
    master = combine_dfs(dfs) 
    master = clean_if_dirty(master) 
    master = mod_types_pct(master) 
    master = process_averages(master) 
    
    master.to_sql('master_'+season_str, con=engine, if_exists='replace', index=False) 
    
    



C:\Users\bchan\AppData\Local\Temp\ipykernel_5768\1912748627.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_df = stat_df.groupby('Player').apply(lambda x: select_team(x)).reset_index(drop=True)
C:\Users\bchan\AppData\Local\Temp\ipykernel_5768\1912748627.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_df = stat_df.groupby('Player').apply(lambda x: select_team(x)).reset_index(drop=True)
C:

# Clean Tables 

In [12]:
engine = sqlalchemy.create_engine('sqlite:///../../DB/ballbase.db') 

start_ssn = 1973   
stop_ssn = 2024    

for season in range(start_ssn, stop_ssn):   
    season_str = str(season) + '_' + str(season+1)[2:] 
    master = pd.read_sql("master"+"_"+season_str, con=engine)  

    # Cleaning code here ... 
            
    master.to_sql('master'+'_'+season_str, con = engine, if_exists = 'replace', index = False)  
        

In [13]:
pd.read_sql('master_1993_94', con=engine).sort_values(by='APG', ascending=False).head(10)   

,Player,Pos,Age,Team,GP,GS,MPG,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORPG,DRPG,RPG,APG,SPG,BPG,ToPG,PF,PPG,T2P,T2PA,T3P,T3PA,TAST,TBLK,TDRB,TFG,TFGA,TFT,TFTA,TMP,TORB,TPF,TPTS,TSTL,TTOV,TTRB,DRtg,ORtg,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48
203,John Stockton,PG,31.0,UTA,82.0,82.0,36.2,5.6,10.6,52.8,0.6,1.8,32.2,5.0,8.8,57.0,55.5,3.3,4.1,80.5,0.9,2.3,3.1,12.6,2.4,0.3,3.2,2.9,15.1,410.0,719.0,48.0,149.0,1031.0,22.0,186.0,458.0,868.0,272.0,338.0,2969.0,72.0,236.0,1236.0,199.0,266.0,258.0,104.0,121.0,0.172,53.1,0.5,8.0,2.2,7.1,4.0,0.389,5.8,2.8,9.3,22.5,3.5,20.7,5.0,60.8,19.1,7.5,13.2,0.214
281,Muggsy Bogues,PG,29.0,CHH,77.0,77.0,35.7,4.6,9.8,47.1,0.0,0.2,16.7,4.6,9.6,47.6,47.3,1.6,2.0,80.6,1.0,3.1,4.1,10.1,1.7,0.0,2.2,1.9,10.8,352.0,739.0,2.0,12.0,780.0,2.0,235.0,354.0,751.0,125.0,155.0,2746.0,78.0,147.0,835.0,133.0,171.0,313.0,110.0,114.0,0.016,39.2,0.0,1.1,-0.4,9.2,1.7,0.206,1.4,3.2,4.8,16.2,2.4,17.3,6.3,51.0,15.4,2.1,6.4,0.112
278,Mookie Blaylock,PG,26.0,ATL,81.0,81.0,36.0,5.5,13.3,41.1,1.4,4.2,33.4,4.1,9.1,44.7,46.4,1.4,2.0,73.0,1.4,3.8,5.2,9.7,2.6,0.5,2.4,1.8,13.8,330.0,738.0,114.0,341.0,789.0,44.0,307.0,444.0,1079.0,116.0,159.0,2915.0,117.0,144.0,1118.0,212.0,196.0,424.0,100.0,111.0,0.316,40.4,1.0,5.7,2.3,11.6,5.3,0.147,3.4,4.4,4.9,19.6,3.7,14.6,8.0,48.7,19.8,5.7,10.2,0.168
217,Kenny Anderson,PG,23.0,NJN,82.0,82.0,38.2,7.0,16.8,41.7,0.5,1.6,30.3,6.5,15.2,42.9,43.2,4.2,5.2,81.8,1.1,2.8,3.9,9.6,1.9,0.2,3.2,2.5,18.8,536.0,1249.0,40.0,132.0,784.0,15.0,233.0,576.0,1381.0,346.0,423.0,3135.0,89.0,201.0,1538.0,158.0,266.0,322.0,106.0,106.0,0.096,40.6,0.3,2.4,-0.1,8.0,3.2,0.306,2.5,2.9,4.3,18.2,2.5,14.5,5.4,49.1,24.6,3.4,7.5,0.114
226,Kevin Johnson,PG,27.0,PHO,67.0,67.0,36.6,7.1,14.6,48.7,0.1,0.4,22.2,7.0,14.2,49.4,49.0,5.7,6.9,81.9,0.8,1.7,2.5,9.5,1.9,0.1,3.5,1.9,20.0,471.0,953.0,6.0,27.0,637.0,10.0,112.0,477.0,980.0,380.0,464.0,2449.0,55.0,127.0,1340.0,125.0,235.0,167.0,109.0,116.0,0.028,38.5,0.3,3.4,-0.7,5.1,1.7,0.473,4.1,2.6,7.0,20.6,2.5,16.6,3.8,56.6,24.3,3.4,8.7,0.170
319,Rod Strickland,PG,27.0,POR,82.0,58.0,35.2,6.4,13.3,48.3,0.0,0.1,20.0,6.4,13.2,48.6,48.4,4.3,5.7,74.9,1.5,3.0,4.5,9.0,1.8,0.3,3.1,2.1,17.2,526.0,1083.0,2.0,10.0,740.0,24.0,248.0,528.0,1093.0,353.0,471.0,2889.0,122.0,171.0,1411.0,147.0,257.0,370.0,106.0,113.0,0.009,38.2,0.5,3.6,0.5,9.8,3.0,0.431,3.1,4.4,6.3,19.8,2.5,16.5,7.0,54.3,22.0,4.1,9.3,0.155
345,Sherman Douglas,PG,27.0,BOS,78.0,78.0,35.8,5.4,11.8,46.2,0.2,0.7,23.2,5.3,11.1,47.7,47.0,2.3,3.5,64.1,0.9,1.6,2.5,8.8,1.1,0.1,3.0,2.2,13.3,412.0,863.0,13.0,56.0,683.0,11.0,123.0,425.0,919.0,177.0,276.0,2789.0,70.0,171.0,1040.0,89.0,233.0,193.0,113.0,103.0,0.061,35.8,0.2,-1.4,-2.2,5.0,0.8,0.300,0.8,2.8,1.9,14.3,1.6,18.3,3.9,50.0,19.8,0.4,2.7,0.046
259,Mark Jackson,PG,28.0,LAC,79.0,79.0,34.3,4.2,9.3,45.2,0.5,1.6,28.3,3.7,7.7,48.8,47.7,2.1,2.7,79.1,1.4,3.1,4.4,8.6,1.5,0.1,2.9,1.5,10.9,295.0,605.0,36.0,127.0,678.0,6.0,241.0,331.0,732.0,167.0,211.0,2711.0,107.0,115.0,865.0,120.0,232.0,348.0,111.0,107.0,0.173,34.6,0.1,-0.1,-0.8,9.4,1.4,0.288,0.7,4.2,2.7,14.8,2.1,22.0,6.8,52.4,16.1,1.3,4.1,0.072
261,Mark Price,PG,29.0,CLE,76.0,73.0,31.4,6.3,13.2,47.8,1.6,3.9,39.7,4.8,9.3,51.1,53.6,3.1,3.5,88.8,0.5,2.5,3.0,7.8,1.4,0.1,2.5,1.2,17.3,362.0,708.0,118.0,297.0,589.0,11.0,189.0,480.0,1005.0,238.0,268.0,2386.0,39.0,93.0,1316.0,103.0,189.0,228.0,107.0,118.0,0.296,42.0,0.3,6.4,0.1,9.2,2.4,0.267,6.3,1.9,7.6,22.7,2.3,14.4,5.6,58.6,24.6,5.0,10.0,0.201
272,Micheal Williams,PG,27.0,MIN,71.0,66.0,31.1,4.4,9.7,45.7,0.1,0.6,22.2,4.3,9.0,47.4,46.4,4.7,5.6,83.9,0.9,2.2,3.1,7.2,1.7,0.3,2.9,2.7,13.7,304.0,642.0,10.0,45.0,512.0,24.0,154.0,314.0,687.0,333.0,397.0,2206.0,67.0,193.0,971.0,118.0,203.0,221.0,109.0,111.0,0.066,37.8,0.7,1.9,-0.1,8.0,1.5,0.578,2.0,3.6,4.1,18.0,2.7,19.1,5.9,56.3,21.1,2.2,5.6,0.122
